In [15]:
# This script generates deep learning training data
# Author: Nikita Mishra
# run: python Deep_learning_training_data.py

# TODO: remove repeated solutions and change start and end position accordingly
import sys
import json
import pandas as pd
import numpy as np
import os
from difflib import SequenceMatcher
from bs4 import BeautifulSoup
from pathlib import Path
from importlib.machinery import SourceFileLoader
import api_client
from api_client import *
import random


#api_client = SourceFileLoader("module.name", "../api_client.py").load_module()

DATA_DIR = 'data/'
SOLUTIONS_FILE = ''.join([DATA_DIR, 'solutions_org'])
QUERIES_FILE   = ''.join([DATA_DIR, 'queries_org'])
CAUCUSES_FILE  = ''.join([DATA_DIR, 'caucuses_org'])

MAX_OUTPUT = 1000000

MAX_LENGTH = 400
orgs_api = ApiClient('http://localhost:3002', '/v1/orgs')
def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    return

def read(filename):
    print('Loading from file ', filename, '...')
    with open(filename) as data_file:
        return data_file.read().splitlines()

def length_distribution( solutions_caucus):
    return [len(u.split(' ')) for u in solutions_caucus]

def print_dist(x):
    print('min:',min(x),'mean:',sum(x)/len(x),'max:',max(x))

    #TODO: Fix split
def split_query(queries, solutions):
    print('Inside split queries')
    
    print(len(queries),len(solutions))
    new_queries, new_solutions, old_id = [],[],[]
    for i,(query,solution) in enumerate(zip(queries, solutions)):
        query_split = query.split(" ")
        start = 0
        for item in range(10,len(query_split),10):

            new_queries.append(' '.join(query_split[start:item]))
            new_solutions.append(solution)
            old_id.append(i)
            start = item
    print(len(new_queries), len(new_solutions))
    return new_queries, new_solutions, old_id

In [61]:
# Bagging
from joblib import Parallel, delayed
import multiprocessing
from statistics import mean
from statistics import stdev
import math
def classify_sample(LOGITS_positive, LOGITS_negative):    
        ids = random.sample(range(len(LOGITS_positive)),  math.ceil(len(LOGITS_positive)*0.7) ) 
        LOGITS_positive = [LOGITS_positive[item] for item in ids ]
        LOGITS_negative = [LOGITS_negative[item] for item in ids ]

        left_point = min(LOGITS_positive+ LOGITS_negative)
        right_point = max(LOGITS_positive+ LOGITS_negative)
        errors=[]
        num_steps = 100
        gap = int(np.ceil(abs(left_point-right_point)/(num_steps - 1)))
        if gap <=0:
            print('Left should be smaller than right')
            print('Left:', left_point, ', Right:', right_point)
            return
        for threshold in np.arange(left_point, right_point, gap):
            error_positive = sum(i > threshold for i in LOGITS_positive) 
            error_negative = sum(i < threshold for i in LOGITS_negative) 
            error = error_positive + error_negative
            errors.append(error)
            #print(threshold, error, error_positive, error_negative)
        return min(errors)/(len(LOGITS_negative)+len(LOGITS_positive))
def classify_bagged(LOGITS_positive, LOGITS_negative):
    
    num_cores = multiprocessing.cpu_count()
    results = Parallel(n_jobs=num_cores)(delayed(classify_sample)(LOGITS_positive, LOGITS_negative) for i in range(10))
    print(mean(results),stdev(results))

In [88]:
queries_caucus_positive = read('/Users/nikita/nmt_data/Upwork/caucus_positive/sources.tok')
solutions_caucus_positive = read('/Users/nikita/nmt_data/Upwork/caucus_positive/targets.tok')
queries_caucus_negative = read('/Users/nikita/nmt_data/Upwork/caucus_negative/sources.tok')
solutions_caucus_negative = read('/Users/nikita/nmt_data/Upwork/caucus_negative/targets.tok')

Loading from file  /Users/nikita/nmt_data/Upwork/caucus_positive/sources.tok ...
Loading from file  /Users/nikita/nmt_data/Upwork/caucus_positive/targets.tok ...
Loading from file  /Users/nikita/nmt_data/Upwork/caucus_negative/sources.tok ...
Loading from file  /Users/nikita/nmt_data/Upwork/caucus_negative/targets.tok ...


In [18]:
new_queries_positive, new_solutions_positive, id_positive = split_query(queries_caucus_positive, solutions_caucus_positive)
new_queries_negative, new_solutions_negative, id_negative = split_query(queries_caucus_negative, solutions_caucus_negative)

Inside split queries
1382 1382
4867 4867
Inside split queries
1382 1382
4867 4867


In [24]:
pd.DataFrame(new_queries_positive).to_csv('/Users/nikita/nmt_data/Upwork/caucus_positive_split/sources.txt', index=False,header=False)
pd.DataFrame(new_solutions_positive).to_csv('/Users/nikita/nmt_data/Upwork/caucus_positive_split/targets.txt', index=False,header=False)
pd.DataFrame(new_queries_negative).to_csv('/Users/nikita/nmt_data/Upwork/caucus_negative_split/sources.txt', index=False,header=False)
pd.DataFrame(new_solutions_negative).to_csv('/Users/nikita/nmt_data/Upwork/caucus_negative_split/targets.txt', index=False,header=False)

In [26]:
!bash /Users/nikita/Desktop/seq2seq/bin/data/tokenize.sh Upwork/caucus_positive_split
!bash /Users/nikita/Desktop/seq2seq/bin/data/tokenize.sh Upwork/caucus_negative_split
#!scp -r ~/nmt_data/Upwork/caucus_* deep_machine:~/nmt_data/Upwork/

Writing to /Users/nikita/nmt_data/Upwork/caucus_positive_split. To change this, set the OUTPUT_DIR environment variable.
Tokenizing /Users/nikita/nmt_data/Upwork/caucus_positive_split/sources.txt...
Tokenizing /Users/nikita/nmt_data/Upwork/caucus_positive_split/targets.txt...
/Users/nikita/Desktop/seq2seq/bin/data/tokenize.sh: line 51: /Users/nikita/nmt_data/Upwork/caucus_positive_split/train/sources.tok: No such file or directory
Writing to /Users/nikita/nmt_data/Upwork/caucus_negative_split. To change this, set the OUTPUT_DIR environment variable.
Tokenizing /Users/nikita/nmt_data/Upwork/caucus_negative_split/sources.txt...
Tokenizing /Users/nikita/nmt_data/Upwork/caucus_negative_split/targets.txt...
/Users/nikita/Desktop/seq2seq/bin/data/tokenize.sh: line 51: /Users/nikita/nmt_data/Upwork/caucus_negative_split/train/sources.tok: No such file or directory


# Results v2

In [75]:
positive_scores = read('/Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_positive_sources_split-v2-grep.txt')
negative_scores = read('/Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_negative_sources_split-v2-grep.txt')
positive_scores = [float(item.split()[2]) for item in positive_scores]
negative_scores = [float(item.split()[2]) for item in negative_scores]
positive_likelihood = []
for i,j in enumerate(id_positive):
    if j < len(positive_likelihood):
        positive_likelihood[j] = positive_likelihood[j] + positive_scores[i]
    else:
        positive_likelihood.append(positive_scores[i])
negative_likelihood = []
for i,j in enumerate(id_positive):
    if j < len(negative_likelihood):
        negative_likelihood[j] = negative_likelihood[j] + negative_scores[i]
    else:
        negative_likelihood.append(negative_scores[i])

print_dist(positive_likelihood)
print_dist(negative_likelihood)
errors = classify_bagged(negative_likelihood, positive_likelihood)

Loading from file  /Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_positive_sources_split-v2-grep.txt ...
Loading from file  /Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_negative_sources_split-v2-grep.txt ...
min: 84.9566 mean: 1662.6893208393642 max: 42792.790000000015
min: 56.8234 mean: 1717.8845462373356 max: 34738.38
0.492665289256 0.002990123050425948


In [76]:
positive_scores = read('/Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_positive_sources_split-grep.txt')
negative_scores = read('/Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_negative_sources_split-grep.txt')

positive_scores = [float(item.split()[2]) for item in positive_scores]
negative_scores = [float(item.split()[2]) for item in negative_scores]
positive_likelihood = []
for i,j in enumerate(id_positive):
    if j < len(positive_likelihood):
        positive_likelihood[j] = positive_likelihood[j] + positive_scores[i]
    else:
        positive_likelihood.append(positive_scores[i])
        
negative_likelihood = []
for i,j in enumerate(id_positive):
    if j < len(negative_likelihood):
        negative_likelihood[j] = negative_likelihood[j] + negative_scores[i]
    else:
        negative_likelihood.append(negative_scores[i])
print_dist(positive_likelihood)
print_dist(negative_likelihood)
errors = classify_bagged(negative_likelihood, positive_likelihood)

Loading from file  /Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_positive_sources_split-grep.txt ...
Loading from file  /Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_negative_sources_split-grep.txt ...
min: 91.0314 mean: 1619.6292146164985 max: 42614.29
min: 54.2006 mean: 1670.8372235890056 max: 34351.990000000005
0.493595041322 0.0034452475036586194


# Results without id

In [72]:
positive_scores = read('/Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_positive_sources_split-grep.txt')
negative_scores = read('/Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_negative_sources_split-grep.txt')

positive_scores = [float(item.split()[2]) for item in positive_scores]
negative_scores = [float(item.split()[2]) for item in negative_scores]

errors = classify_bagged(positive_scores, negative_scores)

Loading from file  /Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_positive_sources_split-grep.txt ...
Loading from file  /Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_negative_sources_split-grep.txt ...
0.455972996771 0.003271353780902163


In [77]:
# Results without id v2
positive_scores = read('/Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_positive_sources_split-v2-grep.txt')
negative_scores = read('/Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_negative_sources_split-v2-grep.txt')

positive_scores = [float(item.split()[2]) for item in positive_scores]
negative_scores = [float(item.split()[2]) for item in negative_scores]

errors = classify_bagged(positive_scores, negative_scores)

Loading from file  /Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_positive_sources_split-v2-grep.txt ...
Loading from file  /Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_negative_sources_split-v2-grep.txt ...
0.454285294981 0.004115151908238806


# Results with averaging

In [ ]:
positive_scores = read('/Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_positive_sources_split-v2-grep.txt')
negative_scores = read('/Users/nikita/nmt_data/Upwork/scoring_results/chunked_caucus_negative_sources_split-v2-grep.txt')
positive_scores = [float(item.split()[2]) for item in positive_scores]
negative_scores = [float(item.split()[2]) for item in negative_scores]



negative_likelihood = []
for i,j in enumerate(id_positive):
    if j < len(negative_likelihood):
        negative_likelihood[j] = negative_likelihood[j] + negative_scores[i]
    else:
        negative_likelihood.append(negative_scores[i])

print_dist(positive_likelihood)
print_dist(negative_likelihood)
errors = classify_bagged(negative_likelihood, positive_likelihood)

In [ ]:
def likelihood_averaging(id_positive, positive_likelihood):
    positive_likelihood = []
    for i,j in enumerate(id_positive):
        if j < len(positive_likelihood):
            positive_likelihood[j] = positive_likelihood[j] + positive_scores[i]
        else:
            positive_likelihood.append(positive_scores[i])
            
            

In [85]:
from collections import Counter

a = dict(Counter(id_positive))

(a.keys())        

dict_keys([0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 64, 65, 67, 69, 70, 71, 72, 74, 76, 78, 79, 82, 83, 84, 86, 87, 89, 90, 91, 92, 93, 94, 95, 96, 97, 101, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 153, 154, 156, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 190, 191, 192, 193, 194, 195, 196, 199, 201, 202, 203, 205, 206, 207, 208, 209, 210, 212, 213, 214, 215, 216, 218, 220, 221, 223, 224, 225, 226, 227, 228, 229, 231, 232, 233, 234, 235, 236, 237, 239, 241, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 256, 257, 258, 259, 260

In [86]:
positive_likelihood = []
for i,j in enumerate(id_positive):
    if j < len(positive_likelihood):
        positive_likelihood[j] = positive_likelihood[j] + positive_scores[i]
    else:
        positive_likelihood.append(positive_scores[i])
from collections import Counter

a = dict(Counter(id_positive))

positive_likelihood = [item/a[i] for i,item in enumerate(positive_likelihood)]

KeyError: 6

In [87]:
id_positive

[0,
 0,
 0,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 4,
 4,
 4,
 4,
 4,
 5,
 7,
 8,
 8,
 8,
 9,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 18,
 19,
 19,
 19,
 20,
 20,
 20,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 22,
 24,
 24,
 24,
 24,
 24,
 25,
 25,
 25,
 26,
 26,
 26,
 27,
 27,
 27,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 31,
 32,
 32,
 32,
 32,
 32,
 32,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 34,
 35,
 35,
 36,
 36,
 37,
 37,
 37,
 37,
 37,
 37,
 38,
 38,
 40,
 40,
 40,
 40,
 40,
 41,
 41,
 41,
 41,
 42,
 42,
 43,
 43,
 43,
 43,
 44,
 44,
 44,
 44,
 44,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 46,
 46,
 46,
 46,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 47,
 48,
 48,
 48,
 48,
 48,
 48,


In [93]:
queries_caucus_positive

['&quot; Job Success percentage . Hi Upwork , .. My agency success percentage &amp; individual profile success is same . This is not possible , How can i show individual success percentage for each agency members ? &quot;',
 '&quot; I am wondering why I have a a lot of oDesk readiness tests taken with Failed scores . Please check immediately , a lot of other freelancers are also complaining about the same issue . &quot;',
 '&quot; Sir / Mam , due to some negative feedbacks , i dropped from top rated &amp; above 90 % job score to 77 % now and also lost the top rated tag. so please suggest me that how i can get my top position again . One more thing i want to know that may i apply for job the job posts where client require job score and rising talent ? Please suggest me . &quot;',
 'i cant see new job post on my dashboard . I cant see new job post on my dashboard',
 'Dear sir.I am a new freelancer in upwork . I have a problem to sing in my account . I forget my password and security ques

In [92]:
new_queries_positive, new_solutions_positive, id_positive = split_query(queries_caucus_positive, solutions_caucus_positive)
new_queries_negative, new_solutions_negative, id_negative = split_query(queries_caucus_negative, solutions_caucus_negative)

Inside split queries
1382 1382
4867 4867
Inside split queries
1382 1382
4867 4867


In [96]:
queries_caucus_positive[6]

'Password . Recently resetted password still cannot log in'

In [94]:
new_queries_positive

['&quot; Job Success percentage . Hi Upwork , .. My',
 'agency success percentage &amp; individual profile success is same .',
 'This is not possible , How can i show individual',
 '&quot; I am wondering why I have a a lot',
 'of oDesk readiness tests taken with Failed scores . Please',
 'check immediately , a lot of other freelancers are also',
 '&quot; Sir / Mam , due to some negative feedbacks',
 ', i dropped from top rated &amp; above 90 %',
 'job score to 77 % now and also lost the',
 'top rated tag. so please suggest me that how i',
 'can get my top position again . One more thing',
 'i want to know that may i apply for job',
 'the job posts where client require job score and rising',
 'i cant see new job post on my dashboard .',
 'Dear sir.I am a new freelancer in upwork . I',
 'have a problem to sing in my account . I',
 'forget my password and security question answer to change new',
 'password.I try to change my password but I can not',
 'success . Please help me .. User name